<center> <h1> Deep Learning - MLP & CNN </h1> </center>
<center> <h3><i> With MNIST dataset</i> </h3></center>

## Imports

In [33]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from tqdm import tqdm
from torch import no_grad, optim, max
from torchvision import transforms, datasets
from torch.utils.data import DataLoader

## Data Prep
* Download MNIST datasets
* Create Dataloaders

In [34]:
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])

train_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
test_dataset = datasets.MNIST(root='./data', train=False, download=True, transform=transform)

In [35]:
train_loader = DataLoader(dataset=train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=64, shuffle=False)

## Models
* MLP with 2 layers
* CNN

In [36]:
class MLP(nn.Module):
    def __init__(self):
        super(MLP, self).__init__()
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(28 * 28, 128)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.flatten(x)
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x

In [37]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1)
        self.relu = nn.ReLU()
        self.maxpool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(32 * 14 * 14, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = self.relu(x)
        x = self.maxpool(x)
        x = self.flatten(x)
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x

## MLP
* Train
* Evaluate

In [38]:
mlp = MLP()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(mlp.parameters(), lr=0.001)

epochs = 5

mlp.train()
for epoch in range(epochs):
    for data, target in tqdm(train_loader):
        optimizer.zero_grad()
        output = mlp(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()

100%|██████████| 938/938 [01:00<00:00, 15.61it/s]


In [39]:
nb_right = 0
nb = 0

mlp.eval()
for data, target in tqdm(test_loader):
    output = mlp(data)
    nb_right += torch.sum(torch.argmax(output, axis=1) == target)
    nb += output.shape[0]
    
print(f"MLP model accuracy on test dataset: {100*nb_right/nb:.4f}%")

100%|██████████| 157/157 [00:09<00:00, 16.63it/s]


MLP model accuracy on test dataset: 95.9500%


## CNN
* Train
* Evaluate

In [40]:
cnn = CNN()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(cnn.parameters(), lr=0.001)

epochs = 5

cnn.train()
for epoch in range(epochs):
    for data, target in tqdm(train_loader):
        optimizer.zero_grad()
        output = cnn(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()

100%|██████████| 938/938 [00:58<00:00, 16.14it/s]


In [42]:
nb_right = 0
nb = 0

cnn.eval()
for data, target in tqdm(test_loader):
    output = cnn(data)
    nb_right += torch.sum(torch.argmax(output, axis=1) == target)
    nb += output.shape[0]
    
print(f"CNN model accuracy on test dataset: {100*nb_right/nb:.4f}%")

100%|██████████| 157/157 [00:12<00:00, 12.94it/s]

CNN model accuracy on test dataset: 98.1500%
